#### Date: Jun 2019 (*Review: August 2024*)<br>Programmer:  Patricio López-Serrano, Yiğitcan Özer
<br>This demo illustrates decomposition of a loop-based electronic dance music track, following [1].

#### The notebook proceeds in the following steps:
<br>1. Load audio files for the complete, downmixed track, as well as for the individual loops that the track contains.
<br>2. Compute STFTs for all audio data.
<br>3. Each loop becomes a fixed template ("page") in the tensor W.
<br>The track spectrogram is the target to approximate, V.
<br>We wish to learn the activation matrix H, which answers the question
<br>"Where was each loop activated throughout the track?"
<br>4. Visualize results.

### Initialization

In [ ]:
import os
import numpy as np
import soundfile as sf
import IPython.display as ipd

from libnmfd.core.nmfconv import init_activations, nmfd
from libnmfd.dsp.filters import alpha_wiener_filter
from libnmfd.dsp.transforms import forward_stft, inverse_stft, log_freq_log_mag
from libnmfd.utils import make_monaural, pcm_int16_to_float32np
from libnmfd.utils.core_utils import visualize_components_nmf

INPUT_DIR = 'data/'
OUT_DIR = 'output/'

filename = 'LSDDM_EM_track.wav'
filename_fx = 'LSDDM_EM_Effects.wav'
filename_bass = 'LSDDM_EM_bass.wav'
filename_melody = 'LSDDM_EM_melody.wav'
filename_drums = 'LSDDM_EM_drums.wav'

### 1. Load the audio signal

In [ ]:
x_tr, fs = sf.read(os.path.join(INPUT_DIR, filename))
x_fx, _ = sf.read(os.path.join(INPUT_DIR, filename_fx))
x_bass, _ = sf.read(os.path.join(INPUT_DIR, filename_bass))
x_melody, _ = sf.read(os.path.join(INPUT_DIR, filename_melody))
x_drums, _ = sf.read(os.path.join(INPUT_DIR, filename_drums))

# make monaural if necessary
x_tr = make_monaural(x_tr)
x_fx = make_monaural(x_fx)
x_bass = make_monaural(x_bass)
x_melody = make_monaural(x_melody)
x_drums = make_monaural(x_drums)

### 2. Compute STFT

In [ ]:
# spectral parameters
BLOCK_SIZE = 2048
HOP_SIZE = 512

# STFT computation
X_tr, A_tr, P_tr = forward_stft(x_tr, block_size=BLOCK_SIZE, hop_size=HOP_SIZE, reconst_mirror=True, append_frames=True)

# get dimensions and time and freq resolutions
num_bins, num_frames = X_tr.shape
time_res = HOP_SIZE / fs
freq_res = fs / BLOCK_SIZE

# get logarithmically-spaced frequency axis version for visualization
log_freq_log_mag_A, log_freq_axis = log_freq_log_mag(A_tr, freq_res=freq_res)
num_log_bins = len(log_freq_axis)

# repeat for FX loop ---------------------------------------------------
X_bass, A_bass, _ = forward_stft(x_bass, block_size=BLOCK_SIZE, hop_size=HOP_SIZE, reconst_mirror=True, append_frames=True)
X_fx, A_fx, _ = forward_stft(x_fx, block_size=BLOCK_SIZE, hop_size=HOP_SIZE, reconst_mirror=True, append_frames=True)
X_melody, A_melody, _ = forward_stft(x_melody, block_size=BLOCK_SIZE, hop_size=HOP_SIZE, reconst_mirror=True, append_frames=True)
X_drums, A_drums, _ = forward_stft(x_drums, block_size=BLOCK_SIZE, hop_size=HOP_SIZE, reconst_mirror=True, append_frames=True)
num_bins_bass, num_frames_bass = X_bass.shape

### 3. Apply NMF variants to STFT magnitude

In [ ]:
# set common parameters
num_comp = 4
num_iter = 30
num_template_frames = num_frames_bass

init_W = list()
init_W.append(A_drums)
init_W.append(A_melody)
init_W.append(A_bass)
init_W.append(A_fx)

# generate initial activations
init_H = init_activations(num_comp=num_comp,
                          num_frames=num_frames,
                          strategy='uniform')

In [ ]:
# NMFD core method
nmfd_W, nmfd_H, nmfd_V, _, _ = nmfd(V=A_tr, 
                                    num_comp=num_comp, 
                                    num_frames=num_frames, 
                                    num_bins=num_bins,
                                    num_iter=num_iter,
                                    num_template_frames=num_template_frames,
                                    init_W=init_W,
                                    init_H=init_H,
                                    fix_W=True)
# alpha-Wiener filtering
nmfd_A, _ = alpha_wiener_filter(A_tr, nmfd_V, 1.0)

In [ ]:
# visualize
fh1, _ = visualize_components_nmf(V=A_tr, W=nmfd_W, H=nmfd_H, comp_V=nmfd_V, time_res=time_res,
                                  freq_res=freq_res, log_comp=1e5, font_size=14)

fh1.savefig(os.path.join(OUT_DIR, 'LSDDM_EM.png'))

#### References 
[1] Patricio López-Serrano, Christian Dittmar, Jonathan Driedger, and Meinard Müller.
<br>**Towards Modeling and Decomposing Loop-based Electronic Music**
<br>In Proceedings of the International Conference on Music Information Retrieval (ISMIR), pages 502–508, New York City, USA, August 2016.

#### If you use the 'NMF toolbox' please refer to:
[2] Patricio López-Serrano, Christian Dittmar, Yiğitcan Özer, and Meinard Müller<br>
**NMF Toolbox: Music Processing Applications of Nonnegative Matrix Factorization**<br>
In Proceedings of the  International Conference on Digital Audio Effects (DAFx), 2019.